# Import Packages

In [ ]:
import datetime
import seaborn as sns
import quantities as pq
import neurotic
from modules.plot_utils import plot_signals_with_axes

pq.markup.config.use_unicode = True  # allow symbols like mu for micro in output
pq.mN = pq.UnitQuantity('millinewton', pq.N/1e3, symbol = 'mN');  # define millinewton

# IPython Magics

In [ ]:
# make figures interactive and open in a separate window
# %matplotlib qt

# make figures interactive and inline
%matplotlib notebook

# make figures non-interactive and inline
# %matplotlib inline

# Data Parameters

## For ShowCASE 2019 poster...

In [ ]:
# Jade's colors
i2_color  = '#0072BD' # MATLAB blue
rn_color  = '#D95319' # MATLAB orange
bn2_color = '#EDB120' # MATLAB yellow
bn3_color = '#7E2F8E' # MATLAB purple
force_color = '#666666' # dark gray

kwargs = dict(
    figsize = (14, 7),
    majorticks = 5,
    minorticks = 1,
    ylabel_offset = -0.06,
)

In [ ]:
# FRESH FOOD

outfile_basename = 'traces-fresh-food'
data_set_name = 'IN VIVO / JG08 / 2018-06-25 / 001'
t_start, t_stop = [1334, 1364] * pq.s # 30 sec
plots = [
    {'channel': 'I2',    'units': 'uV', 'ylim': [ -20,  20], 'decimation_factor':  10, 'color': i2_color},
    {'channel': 'RN',    'units': 'uV', 'ylim': [-150, 150], 'decimation_factor':  10, 'color': rn_color},
    {'channel': 'BN2',   'units': 'uV', 'ylim': [-180, 180], 'decimation_factor':  10, 'color': bn2_color},
    {'channel': 'BN3',   'units': 'uV', 'ylim': [-120, 120], 'decimation_factor':  10, 'color': bn3_color},
    {'channel': 'Force', 'units': 'mN', 'ylim': [ -10, 300], 'decimation_factor': 100, 'color': force_color},
]

In [ ]:
# TWO-PLY NORI

outfile_basename = 'traces-two-ply-nori'
data_set_name = 'IN VIVO / JG08 / 2018-06-25 / 001'
t_start, t_stop = [3262, 3292] * pq.s # 30 sec
plots = [
    {'channel': 'I2',    'units': 'uV', 'ylim': [ -20,  20], 'decimation_factor':  10, 'color': i2_color},
    {'channel': 'RN',    'units': 'uV', 'ylim': [-150, 150], 'decimation_factor':  10, 'color': rn_color},
    {'channel': 'BN2',   'units': 'uV', 'ylim': [-180, 180], 'decimation_factor':  10, 'color': bn2_color},
    {'channel': 'BN3',   'units': 'uV', 'ylim': [-120, 120], 'decimation_factor':  10, 'color': bn3_color},
    {'channel': 'Force', 'units': 'mN', 'ylim': [ -10, 300], 'decimation_factor': 100, 'color': force_color},
]

In [ ]:
# TAPE NORI

outfile_basename = 'traces-tape-nori'
data_set_name = 'IN VIVO / JG08 / 2018-06-21 / 002'
t_start, t_stop = [175, 205] * pq.s # 30 sec
plots = [
    {'channel': 'I2',    'units': 'uV', 'ylim': [ -30,  30], 'decimation_factor':  10, 'color': i2_color},
    {'channel': 'RN',    'units': 'uV', 'ylim': [ -60,  60], 'decimation_factor':  10, 'color': rn_color},
    {'channel': 'BN2',   'units': 'uV', 'ylim': [-120, 120], 'decimation_factor':  10, 'color': bn2_color},
    {'channel': 'BN3',   'units': 'uV', 'ylim': [-150, 150], 'decimation_factor':  10, 'color': bn3_color},
    {'channel': 'Force', 'units': 'mN', 'ylim': [ -10, 300], 'decimation_factor': 100, 'color': force_color},
]

In [ ]:
# REGULAR NORI

outfile_basename = 'traces-regular-nori'
data_set_name = 'IN VIVO / JG08 / 2018-06-21 / 001'
t_start, t_stop = [2468, 2498] * pq.s # 30 sec
plots = [
    {'channel': 'I2',    'units': 'uV', 'ylim': [ -30,  30], 'decimation_factor':  10, 'color': i2_color},
    {'channel': 'RN',    'units': 'uV', 'ylim': [ -60,  60], 'decimation_factor':  10, 'color': rn_color},
    {'channel': 'BN2',   'units': 'uV', 'ylim': [-120, 120], 'decimation_factor':  10, 'color': bn2_color},
    {'channel': 'BN3',   'units': 'uV', 'ylim': [-150, 150], 'decimation_factor':  10, 'color': bn3_color},
    {'channel': 'Force', 'units': 'mN', 'ylim': [ -10, 300], 'decimation_factor': 100, 'color': force_color},
]

## For August 2019 committee meeting...

In [ ]:
# BRAIN Initiative Grant 2019 colors
i2_color  = '#FFAF14' # orange
rn_color  = '#00CC64' # green
bn2_color = '#4A72FF' # blue
bn3_color = '#7E2F8E' # purple
force_color = '#666666' # dark gray

kwargs = dict(
    figsize = (8, 3),
    majorticks = 20,
    minorticks = 10,
    ylabel_offset = -0.08,
)

In [ ]:
# BRANCHY

outfile_basename = 'force-branchy'
data_set_name = 'IN VIVO / JG17 / 2019-08-20 / 004'
t_start, t_stop = [196, 296] * pq.s # 100 sec
plots = [
#     {'channel': 'BN2',   'units': 'uV', 'ylim': [ -60,  60], 'decimation_factor':  10, 'color': bn2_color},
#     {'channel': 'BN3',   'units': 'uV', 'ylim': [ -60,  60], 'decimation_factor':  10, 'color': bn3_color},
    {'channel': 'Force', 'units': 'mN', 'ylim': [ -10, 350], 'decimation_factor': 100, 'color': force_color},
]

In [ ]:
# TWO-PLY NORI

outfile_basename = 'force-two-ply'
data_set_name = 'IN VIVO / JG17 / 2019-08-20 / 004'
t_start, t_stop = [2689, 2789] * pq.s # 100 sec
plots = [
#     {'channel': 'BN2',   'units': 'uV', 'ylim': [ -60,  60], 'decimation_factor':  10, 'color': bn2_color},
#     {'channel': 'BN3',   'units': 'uV', 'ylim': [ -60,  60], 'decimation_factor':  10, 'color': bn3_color},
    {'channel': 'Force', 'units': 'mN', 'ylim': [ -10, 350], 'decimation_factor': 100, 'color': force_color},
]

In [ ]:
# TAPE NORI

outfile_basename = 'force-tape-nori'
data_set_name = 'IN VIVO / JG17 / 2019-08-17 / 001'
t_start, t_stop = [3532, 3632] * pq.s # 100 sec
plots = [
#     {'channel': 'BN2',   'units': 'uV', 'ylim': [ -60,  60], 'decimation_factor':  10, 'color': bn2_color},
#     {'channel': 'BN3',   'units': 'uV', 'ylim': [ -60,  60], 'decimation_factor':  10, 'color': bn3_color},
    {'channel': 'Force', 'units': 'mN', 'ylim': [ -10, 350], 'decimation_factor': 100, 'color': force_color},
]

# Plot

__IMPORTANT:__ Remember that the large number of points present in the vector graphics output formats (SVG, PDF) of these traces tend to bring most programs, including poster printer software, to a grinding halt. It is recommended for most applications, especially poster printing, to use a high resolution PNG or, if you prefer vector fonts, a combination of rasterized traces and vector labels constructed in Inkscape or Illustrator from a combination of PNG and SVG files.

In [ ]:
# load the data
metadata = neurotic.MetadataSelector('../../data/metadata.yml')
metadata.select(data_set_name)
blk = neurotic.load_dataset(metadata)

In [ ]:
# general plot settings
sns.set(
#     context = 'poster',
    style = 'ticks',
    font_scale = 1,
    font = 'Palatino Linotype',
)

# with sns.plotting_context('poster', font_scale=0.5):
with sns.plotting_context('notebook', font_scale=1):
    start = datetime.datetime.now()
    plot_signals_with_axes(blk, t_start, t_stop, plots, outfile_basename, **kwargs)
    end = datetime.datetime.now()
    print('render time:', end-start)